##  Next Word Prediction Using LSTM
#### Project Overview:

This project aims to develop a deep learning model for predicting the next word in a given sequence of words. The model is built using Long Short-Term Memory (LSTM) networks, which are well-suited for sequence prediction tasks. The project includes the following steps:

1- Data Collection: We use the text of Shakespeare's "Hamlet" as our dataset. This rich, complex text provides a good challenge for our model.

2- Data Preprocessing: The text data is tokenized, converted into sequences, and padded to ensure uniform input lengths. The sequences are then split into training and testing sets.

3- Model Building: An LSTM model is constructed with an embedding layer, two LSTM layers, and a dense output layer with a softmax activation function to predict the probability of the next word.

4- Model Training: The model is trained using the prepared sequences, with early stopping implemented to prevent overfitting. Early stopping monitors the validation loss and stops training when the loss stops improving.

5- Model Evaluation: The model is evaluated using a set of example sentences to test its ability to predict the next word accurately.

6- Deployment: A Streamlit web application is developed to allow users to input a sequence of words and get the predicted next word in real-time.

In [ ]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import  pandas as pd

## load the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')
## save to a file
with open('/content/next_predicter/hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
# Data Preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#load the dataset
with open ('/content/next_predicter/hamlet.txt') as file:
    text = file.read().lower()

#Tokenizer the txt
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

#totalwords
total_words = len(tokenizer.word_index)+1
total_words

4818

In [ ]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'i': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'it': 9,
 'in': 10,
 'that': 11,
 'ham': 12,
 'is': 13,
 'not': 14,
 'his': 15,
 'this': 16,
 'with': 17,
 'your': 18,
 'but': 19,
 'for': 20,
 'me': 21,
 'lord': 22,
 'as': 23,
 'what': 24,
 'he': 25,
 'be': 26,
 'so': 27,
 'him': 28,
 'haue': 29,
 'king': 30,
 'will': 31,
 'no': 32,
 'our': 33,
 'we': 34,
 'on': 35,
 'are': 36,
 'if': 37,
 'all': 38,
 'then': 39,
 'shall': 40,
 'by': 41,
 'thou': 42,
 'come': 43,
 'or': 44,
 'hamlet': 45,
 'good': 46,
 'do': 47,
 'hor': 48,
 'her': 49,
 'let': 50,
 'now': 51,
 'thy': 52,
 'how': 53,
 'more': 54,
 'they': 55,
 'from': 56,
 'enter': 57,
 'at': 58,
 'was': 59,
 'oh': 60,
 'like': 61,
 'most': 62,
 'there': 63,
 'well': 64,
 'know': 65,
 'selfe': 66,
 'would': 67,
 'them': 68,
 'loue': 69,
 'may': 70,
 "'tis": 71,
 'vs': 72,
 'sir': 73,
 'qu': 74,
 'which': 75,
 'did': 76,
 'why': 77,
 'laer': 78,
 'giue': 79,
 'thee': 80,
 'ile': 81,
 'must': 82,
 'hath': 

In [ ]:
#create input sequence
input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sque = token_list[: i+1]
    input_sequences.append(n_gram_sque)

In [ ]:
input_sequences[:10]

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891]]

In [ ]:
# pad sequence
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [ ]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
input_sequences[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1, 687,
         4], dtype=int32)

In [ ]:
#create predictor and label
import tensorflow as tf
x,y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes = total_words)

In [ ]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
#Train and test the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

#Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len-1))
model.add(LSTM(150, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
# model.add(Dense(total_words/2, activation = 'relu'))
model.add(Dense(total_words, activation = 'softmax'))


#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(x_train, y_train, epochs = 100, validation_data = (x_test, y_test), verbose = 1)
history

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 57s 83ms/step - accuracy: 0.0314 - loss: 7.1702 - val_accuracy: 0.0381 - val_loss: 6.6829
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 49s 75ms/step - accuracy: 0.0410 - loss: 6.4535 - val_accuracy: 0.0412 - val_loss: 6.7325
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 82s 76ms/step - accuracy: 0.0474 - loss: 6.2921 - val_accuracy: 0.0488 - val_loss: 6.7765
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 48s 75ms/step - accuracy: 0.0501 - loss: 6.1560 - val_accuracy: 0.0509 - val_loss: 6.7909
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 50s 78ms/step - accuracy: 0.0524 - loss: 6.0172 - val_accuracy: 0.0548 - val_loss: 6.8562
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 50s 77ms/step - accuracy: 0.0629 - loss: 5.8899 - val_accuracy: 0.0655 - val_loss: 6.8792
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 52s 81ms/step - accuracy: 0.0739 - loss: 5.7211 - val_accuracy: 0.0647 - val_loss: 6.9114
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 50s 78ms/step - accuracy: 0.0824 - loss: 5

In [ ]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [ ]:
input_text="To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:To be or not to be
Next Word PRediction:that


In [ ]:
input_text="Osr. I commend my duty to your"
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Next Word PRediction:lordship


In [ ]:
## Define the model
from tensorflow.keras.layers import GRU
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words,activation="softmax"))

# #Compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history3 = model.fit(x_train, y_train, epochs = 50, validation_data = (x_test, y_test), verbose = 1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 51s 69ms/step - accuracy: 0.0303 - loss: 7.2068 - val_accuracy: 0.0334 - val_loss: 6.7768
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 45s 70ms/step - accuracy: 0.0406 - loss: 6.4659 - val_accuracy: 0.0493 - val_loss: 6.7254
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 45s 70ms/step - accuracy: 0.0576 - loss: 6.1799 - val_accuracy: 0.0657 - val_loss: 6.6586
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 81s 68ms/step - accuracy: 0.0688 - loss: 5.8975 - val_accuracy: 0.0732 - val_loss: 6.7037
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 45s 70ms/step - accuracy: 0.0829 - loss: 5.5911 - val_accuracy: 0.0740 - val_loss: 6.7315
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 48s 75ms/step - accuracy: 0.0989 - loss: 5.2848 - val_accuracy: 0.0705 - val_loss: 6.8392
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 78s 69ms/step - accuracy: 0.1096 - loss: 5.0139 - val_accuracy: 0.0736 - val_loss: 6.9850
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 81s 68ms/step - accuracy: 0.1288 - loss: 4.7418 - 